In [1]:
# !pip install openai

In [1]:
import pandas as pd
import json
import openai
import numpy as np
import re

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
pd.set_option('max_colwidth', None)
pd.set_option('display.max_columns', None)

In [4]:
chat = pd.read_parquet('chat_history.parquet')

In [5]:
#pd.read_parquet('embedding.parquet')

In [6]:
chat.head(1)

,processedUtterance,utterance,semantics,sqlForm,status,dataset,tags,createdAt,length
0,Customer_Monthly_Smart_View.totalprice,"""totalprice""","{'domain': 'Metric', 'filter': [], 'columns': ['Customer_Monthly_Smart_View.totalprice'], 'type': None, 'select': [('sum', 'currency/metrics/Customer_Monthly_Smart_View.totalprice')]}","SELECT round(sum(coalesce(subquery.""label_Customer_Monthly_Smart_View.totalprice"", 0)), 2) AS ""label_Customer_Monthly_Smart_View.totalprice"" \nFROM (SELECT round(sum(coalesce(@Customer_Monthly_Smart_View.totalprice, 0)), 2) AS ""label_Customer_Monthly_Smart_View.totalprice"" \nFROM (#)Customer_Monthly_Smart_View) AS subquery",success,3740fcb3-de4f-42cd-a14d-caba82b1cbcc,"[[""$Metrics"", [""\""totalprice\""""]]]",1675057540502,1


In [7]:
chat = chat[['utterance','tags']]

In [8]:
chat

,utterance,tags
0,"""totalprice""","[[""$Metrics"", [""\""totalprice\""""]]]"
1,+ve revenue,"[[""$UOps"", [""+ve""]], [""$Metrics"", [""revenue""]]]"
2,", on_hand_qty, customer, product number, product description, uom code, date","[[""$Metrics"", [""on_hand_qty""]], [""$Dimensions"", [""customer""]], [""$Dimensions"", [""product number""]], [""$Dimensions"", [""product description""]], [""$Dimensions"", [""uom code""]], [""$Datum"", [""date""]]]"
3,-ve revenue,"[[""$UOps"", [""-ve""]], [""$Metrics"", [""revenue""]]]"
4,1)\tlines per day for last 3 months,"[[""$MultiTableDate"", [""1)""]], [""$Dimensions"", [""lines""]], [""$Per"", [""per""]], [""$TrendPeriod"", [""day""]], [""$Filter"", [""for""]], [""$Period"", [""last"", ""3"", ""months""]]]"
...,...,...
3868,ytd spend by vendor top 5,"[[""$Period"", [""ytd""]], [""$Metrics"", [""spend""]], [""$GroupBy"", [""by""]], [""$Dimensions"", [""vendor""]], [""$Super"", [""top""]], [""$Number"", [""5""]]]"
3869,ytd spend by week,"[[""$Period"", [""ytd""]], [""$Metrics"", [""spend""]], [""$GroupBy"", [""by""]], [""$TrendPeriod"", [""week""]]]"
3870,ytd spend this calendar year,"[[""$Period"", [""ytd""]], [""$Metrics"", [""spend""]], [""$Period"", [""this"", ""calendar year""]]]"
3871,ytd spend top 5 vendors,"[[""$Period"", [""ytd""]], [""$Metrics"", [""spend""]], [""$Super"", [""top""]], [""$Number"", [""5""]], [""$Dimensions"", [""vendors""]]]"


In [9]:
#chat['utterance'] = chat['utterance'].str.replace(r'\d+\)', '', regex=True).str.replace('\t|\\\\', '', regex=True)
chat['utterance'] = chat['utterance'].replace(', ', ' ', regex=True).replace('"', '', regex=True).replace('\t|\\\\', '', regex=True)
chat['utterance'] = chat['utterance'].str.strip()

In [10]:
chat['tags'] = chat['tags'].str.replace('" ', '"')
chat['tags'] = chat['tags'].str.strip()

In [11]:
chat

,utterance,tags
0,totalprice,"[[""$Metrics"", [""\""totalprice\""""]]]"
1,+ve revenue,"[[""$UOps"", [""+ve""]], [""$Metrics"", [""revenue""]]]"
2,on_hand_qty customer product number product description uom code date,"[[""$Metrics"", [""on_hand_qty""]], [""$Dimensions"", [""customer""]], [""$Dimensions"", [""product number""]], [""$Dimensions"", [""product description""]], [""$Dimensions"", [""uom code""]], [""$Datum"", [""date""]]]"
3,-ve revenue,"[[""$UOps"", [""-ve""]], [""$Metrics"", [""revenue""]]]"
4,1)lines per day for last 3 months,"[[""$MultiTableDate"", [""1)""]], [""$Dimensions"", [""lines""]], [""$Per"", [""per""]], [""$TrendPeriod"", [""day""]], [""$Filter"", [""for""]], [""$Period"", [""last"", ""3"", ""months""]]]"
...,...,...
3868,ytd spend by vendor top 5,"[[""$Period"", [""ytd""]], [""$Metrics"", [""spend""]], [""$GroupBy"", [""by""]], [""$Dimensions"", [""vendor""]], [""$Super"", [""top""]], [""$Number"", [""5""]]]"
3869,ytd spend by week,"[[""$Period"", [""ytd""]], [""$Metrics"", [""spend""]], [""$GroupBy"", [""by""]], [""$TrendPeriod"", [""week""]]]"
3870,ytd spend this calendar year,"[[""$Period"", [""ytd""]], [""$Metrics"", [""spend""]], [""$Period"", [""this"", ""calendar year""]]]"
3871,ytd spend top 5 vendors,"[[""$Period"", [""ytd""]], [""$Metrics"", [""spend""]], [""$Super"", [""top""]], [""$Number"", [""5""]], [""$Dimensions"", [""vendors""]]]"


#### creating prompt & completion

In [ ]:
z

In [ ]:
a = []
for col in final_df.columns:
    a.append(f'"{col}": "{{ques["{col}"]}}" ')
a = a[:-1]

In [ ]:
print(', '.join(a))

In [78]:
# Create prompt and completion pairs
pairs = []
for index, ques in chat.iterrows():
    prompt = f"Question: {ques['utterance']}"
    completion = f"Tags: {ques['tags']}"
    pairs.append({"prompt":prompt, "completion":completion})


In [80]:
# Export to JSON file
with open('pc_pairs.json', 'w') as jf:
    json.dump(pairs, jf, indent=1)

In [81]:
with open('pc_pairs.jsonl', 'w') as f:
    for pair in pairs:
        f.write(json.dumps(pair) + '\n')

#### fitting model

In [ ]:
z

In [ ]:
# api_key ="Your-openai-key"
# openai.api_key = api_key

In [ ]:
!openai tools fine_tunes.prepare_data -f 'pc_pairs.jsonl'

In [ ]:
upload_response = openai.File.create(
  file=open('pc_pairs.jsonl', "rb"),
  purpose='fine-tune'
)

upload_response

In [ ]:
file_id = upload_response.id

In [ ]:
fine_tune_response = openai.FineTune.create(training_file=file_id, model="curie")
fine_tune_response

In [ ]:
fine_tuned_model = fine_tune_response.fine_tuned_model
fine_tuned_model

In [ ]:
new_prompt = ""
answer = openai.Completion.create(
  model=fine_tuned_model,
  prompt=new_prompt,
  max_tokens=100,
  temperature=0
)
answer['choices'][0]['text']

## test

In [12]:
# api_key ="Your-openai-key"
# openai.api_key = api_key
# COMPLETIONS_MODEL = "text-davinci-003"

In [24]:
prompt = """Tag the words given in Question and give response as tags. Don't try to answer and use the only words given in the Question. Keep order of words in Tags similar to Question
Question: totalprice,
Tags: [["$Metrics", ["totalprice"]]]

Question: revenue for this year
Tags: [["$Metrics", ["revenue"]], ["$Filter", ["for"]], ["$Period", ["this", "year"]]]

Question: revenue for this year, last year
Tags: [["$Metrics", ["revenue"]], ["$Filter", ["for"]], ["$Period", ["this", "year"]], ["$Period", ["last", "year"]]]

Question: 10% of top supplier by revenue
Tags:  [[\"$MultiTableDate\", [\"10% of\"]], [\"$Super\", [\"top\"]], [\"$Dimensions\", [\"supplier\"]], [\"$GroupBy\", [\"by\"]], [\"$Metrics\", [\"revenue\"]]]

Question: 2018 and 2019 revenue by month
Tags:"""

openai.Completion.create(
    prompt=prompt,
    temperature=0,
    max_tokens=300,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    model=COMPLETIONS_MODEL
)["choices"][0]["text"].strip(" \n")

'[["$Period", ["2018", "2019"]], ["$Metrics", ["revenue"]], ["$GroupBy", ["by"]], ["$Dimensions", ["month"]]]'

In [12]:
# ParaPhrased
input_str = '10% of top supplier by revenue'
prompt = f""" 
#Paraphrase the below query with more naturual language variations create as number of variations and add some words if needed like "can you", "can you get me", "give me"

{input_str}

Paraphrased:
"""

response = openai.Completion.create(
  model=COMPLETIONS_MODEL,
  prompt=prompt,
  temperature=0.7,
  max_tokens=256,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)


In [13]:
response["choices"][0]["text"].strip(" \n")

'1. What are the top 10% of suppliers based on revenue?\n2. Show me the suppliers with the highest revenue.\n3. Can you provide me a list of the top 10% of suppliers based on revenue?\n4. Give me the top suppliers that make up 10% of total revenue.\n5. Can you get me a list of the suppliers that make up 10% of total revenue?\n6. Who are the suppliers that account for 10% of total revenue?'

In [29]:
# list1 = []
# list2 = []
# for input_str in questions[:5]:
#     prompt = f""" 
#     #Paraphrase the below query with more naturual language variations create as number of variations and add some words if needed like "can you", "can you get me", "give me"
#     {input_str}

#     Paraphrased:
#     """

#     response = openai.Completion.create(
#       model=COMPLETIONS_MODEL,
#       prompt=prompt,
#       temperature=0.7,
#       max_tokens=256,
#       top_p=1,
#       frequency_penalty=0,
#       presence_penalty=0
#     )["choices"][0]["text"]
#     list1.append(input_str)
#     list2.append(response)

In [13]:
list1 = []
list2 = []

In [14]:
%%time
import time
import json

questions = chat['utterance'].values

chunked, n = questions[:1000], 100
iter_ = 0

for chunk in [chunked[i * n:(i + 1) * n] for i in range((len(chunked) + n - 1) // n )]:
    for input_str in chunk:
        prompt = f""" 
        #Paraphrase the below query with more naturual language variations create as number of variations and add some words if needed like "can you", "can you get me", "give me"
        {input_str}

        Paraphrased:
        """

        try:
            response = openai.Completion.create(
                model=COMPLETIONS_MODEL,
                prompt=prompt,
                temperature=0.7,
                max_tokens=256,
                top_p=1,
                frequency_penalty=0,
                presence_penalty=0,
                timeout=10
            )["choices"][0]["text"].strip(" \n")
            list1.append(input_str)
            list2.append(response)
            time.sleep(1)
        except openai.error.APIConnectionError:
            print("API request failed. Retrying in 5 seconds...")
            time.sleep(5)
            continue
            
    
    dict_ = []
    for ut, para in zip(list1,list2):
        dict_.append({"utterance":ut, "paraphrase":para})
    with open(f'./paraphrase/paraphrased_batch_{iter_}.json', 'w') as jf:
        json.dump(dict_, jf, indent=1)
    iter_ = iter_+1
    print(f"completed {iter_*100} items")
 

completed 100 items
completed 200 items
completed 300 items
completed 400 items
completed 500 items
completed 600 items
completed 700 items
completed 800 items
completed 900 items
completed 1000 items
CPU times: user 2.63 s, sys: 150 ms, total: 2.78 s
Wall time: 1h 20min 55s


In [15]:
len(list2)

1000

In [27]:
data = chat.head(len(list2))
data['paraphrase'] = list2
data.sample(1)

,utterance,tags,paraphrase
776,fulfilled the revenue,"[[""$Datum"", [""fulfilled""]], [""$Optional"", [""the""]], [""$Metrics"", [""revenue""]]]",1. Can you provide me with the revenue?\n 2. Give me the revenue figures.\n 3. Could you reveal the revenue?\n 4. Do you have the revenue figures?\n 5. What is the revenue total?


In [23]:
# chat['paraphrase'] = list2
# chat.head(1)

In [19]:
# Create prompt and completion pairs
pairs = []
for index, ques in data.iterrows():
    prompt = f"Question: {ques['utterance']}, Paraphrase: {ques['paraphrase']}"
    completion = f"Tags: {ques['tags']}"
    pairs.append({"prompt":prompt, "completion":completion})


In [20]:
# Export to JSON file
with open('pc_pairs.json', 'w') as jf:
    json.dump(pairs, jf, indent=1)

In [21]:
with open('pc_pairs.jsonl', 'w') as f:
    for pair in pairs:
        f.write(json.dumps(pair) + '\n')